# Hipscat-import HSC PDR3 forced co-add catalog

In [1]:
from copy import deepcopy
from pathlib import Path

from dask.distributed import Client
from hats_import.catalog.arguments import ImportArguments
from hats_import.margin_cache.margin_cache_arguments import MarginCacheArguments
from hats_import.pipeline import pipeline_with_client

from hsc_fits_reader import HSCFitsReader

/ocean/projects/phy210048p/malanche/hats-import-pipelines/cenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import dask.config

dask.config.set({
    "distributed.comm.timeouts.connect": "600s",
    "distributed.comm.timeouts.tcp": "600s",
    "distributed.nanny.shutdown-timeout": "600s",
    "distributed.deploy.lost-worker-timeout": "600s",
    "distributed.scheduler.idle-timeout": "600s",
    "distributed.scheduler.no-workers-timeout": "600s",
    "distributed.scheduler.locks.lease-timeout": "600s",
})

In [3]:
HATS_ROOT = Path('/ocean/projects/phy210048p/shared/hats')
HATS_PATH = HATS_ROOT / "catalogs" / "hsc_pdr3"
HSC_PDR3_FITS = HATS_ROOT / "raw/hsc/pdr3/catalog-forced"

RA_COLUMN = "coord_ra"
DEC_COLUMN = "coord_dec"

In [4]:
def pipeline(passband):
    catalog_name = f"hsc_pdr3-forced-{passband}"

    input_file_list = sorted(
        HSC_PDR3_FITS.glob(f"forced_src-HSC-{passband.upper()}*.fits")
    )

    if not (HATS_PATH / catalog_name / "properties").exists():  
        args = ImportArguments(
            sort_columns="id",
            ra_column=RA_COLUMN,
            dec_column=DEC_COLUMN,
            lowest_healpix_order=4,  # It would be something between 3 and 6
            input_file_list=input_file_list,
            file_reader=HSCFitsReader(
                ra_column=RA_COLUMN,
                dec_column=DEC_COLUMN,
                flags_column='flags',
                # we cannot do flags yet, see
                # https://github.com/astronomy-commons/hipscat-import/issues/351
                skip_column_names=['flags'],
            ),
            drop_empty_siblings=True,
            output_artifact_name=catalog_name,
            output_path=HATS_PATH,
            resume=True,
            simple_progress_bar=True,
        )
    
        with Client(n_workers=16, threads_per_worker=1, memory_limit="80GB") as client:
            display(client)
            pipeline_with_client(args, client)

    margin_catalog_name = f"{catalog_name}_10arcs"
    if not (HATS_PATH / margin_catalog_name / "properties").exists():  
        margin_args = MarginCacheArguments(
            input_catalog_path=Path(HATS_PATH) / catalog_name,
            output_path=HATS_PATH,
            margin_threshold=10.0,
            output_artifact_name=margin_catalog_name,
            resume=True,
            simple_progress_bar=True,
            fine_filtering=False,
        )
    
        with Client(n_workers=16, threads_per_worker=1, memory_limit="80GB") as client:
            display(client)
            pipeline_with_client(margin_args, client)

In [5]:
 for passband in "GRIZY":
     pipeline(passband)

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 16
Total threads: 16,Total memory: 1.16 TiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43579,Workers: 16
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 1.16 TiB
Comm: tcp://127.0.0.1:42121,Total threads: 1
Dashboard: http://127.0.0.1:33959/status,Memory: 74.51 GiB
Nanny: tcp://127.0.0.1:40205,



2025-02-27 21:13:39,402 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:45221' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {'map_pixel_shards-a7e1910396bdd8850578c8c346770a42', 'map_pixel_shards-f4785769917cd606498d419c22399052', 'map_pixel_shards-3965e15c2f04cb9d66da2577b30c18b2', 'map_pixel_shards-79f695ada4e03eb9332d3e100f9e721d', 'map_pixel_shards-7334b59ec21c3f6a2764ac7eaeca01b7', 'map_pixel_shards-75386e93a9725aabd3c7f797f6f1926a', 'map_pixel_shards-599c452606591bd99b945c2320210979', 'map_pixel_shards-4b68d510fca077db8ecf67ca2a292d31', 'map_pixel_shards-a6dc4939a4740a680f80c26b6b85daf2', 'map_pixel_shards-58d9de65e8ae8a1df762e1cfce78ccee', 'map_pixel_shards-b61fece8f2804a9ea432b43e799c2a8b', 'map_pixel_shards-48239ffd576f5547216e55d318e5de34', 'map_pixel_shards-5d0d540a8a772f0dfdfdade51b1eb4ce', 'map_pixel_shards-7e8936482b74af539d35ce6299d8488b', 'map_pixel_shards-ff002925348740be8b25c991e9fd737c', 'map_pixel_sha